In [1]:
import pandas as pd
import numpy as np
from pycorenlp import StanfordCoreNLP 

In [5]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [16]:
text = "This time around, they're moving even faster"
# output = nlp.annotate(text, properties={
#   'annotators': 'tokenize,ssplit,pos,depparse,parse',
#   'outputFormat': 'json'
#   })

output = nlp.annotate(text, properties={
  'annotators': 'tokenize,ssplit,pos,depparse',
  'outputFormat': 'json'
  })

In [17]:
output

{'sentences': [{'basicDependencies': [{'dep': 'ROOT',
     'dependent': 7,
     'dependentGloss': 'moving',
     'governor': 0,
     'governorGloss': 'ROOT'},
    {'dep': 'det',
     'dependent': 1,
     'dependentGloss': 'This',
     'governor': 2,
     'governorGloss': 'time'},
    {'dep': 'nmod:tmod',
     'dependent': 2,
     'dependentGloss': 'time',
     'governor': 7,
     'governorGloss': 'moving'},
    {'dep': 'advmod',
     'dependent': 3,
     'dependentGloss': 'around',
     'governor': 2,
     'governorGloss': 'time'},
    {'dep': 'punct',
     'dependent': 4,
     'dependentGloss': ',',
     'governor': 7,
     'governorGloss': 'moving'},
    {'dep': 'nsubj',
     'dependent': 5,
     'dependentGloss': 'they',
     'governor': 7,
     'governorGloss': 'moving'},
    {'dep': 'aux',
     'dependent': 6,
     'dependentGloss': "'re",
     'governor': 7,
     'governorGloss': 'moving'},
    {'dep': 'advmod',
     'dependent': 8,
     'dependentGloss': 'even',
     'governor':